In [1]:
import numpy as np
import os

import pydot
from IPython.display import HTML, SVG, display

from pydrake.common import temp_directory, FindResourceOrThrow
from pydrake.geometry import (
    SceneGraph,
    MeshcatVisualizer, MeshcatVisualizerParams, 
    Role, StartMeshcat
)

from pydrake.math import RigidTransform, RollPitchYaw
from pydrake.multibody.parsing import Parser
from pydrake.multibody.plant import MultibodyPlant
from pydrake.multibody.plant import AddMultibodyPlantSceneGraph
from pydrake.systems.analysis import Simulator
from pydrake.systems.framework import DiagramBuilder
from pydrake.visualization import ModelVisualizer

from manipulation.scenarios import AddMultibodyTriad, AddPackagePaths

In [2]:
meshcat = StartMeshcat()

INFO:drake:Meshcat listening for connections at http://localhost:7002


In [3]:
hand_url = FindResourceOrThrow(
    "drake/manipulation/models/wsg_50_description/sdf/schunk_wsg_50_with_tip.sdf"
)
arm_url = '../robot_models/Planar-v2.sdf'

full_2r_directives = '''
directives: 
- add_model: 
    name: 2R_planar_robot
    file: file:///home/dema/PycharmProjects/robot_manipulator/robot_models/Planar-v2.sdf
    default_joint_positions: 
        q1: [0]
        q2: [0]


- add_model: 
    name: end_effector
    file: file:///home/dema/PycharmProjects/robot_manipulator/lib/python3.10/site-packages/pydrake/common/../lib/../share/drake/manipulation/models/wsg_50_description/sdf/schunk_wsg_50_with_tip.sdf
- add_weld: 
    parent: 2R_planar_robot::l2
    child: end_effector::body_frame
    X_PC: 
        translation: [0.5, 0, 0]
        rotation: !Rpy {deg: [0, 0, -90]}
'''

builder = DiagramBuilder()
time_step = 0.003
plant, scene_graph = AddMultibodyPlantSceneGraph(builder, time_step)

parser = Parser(plant)
#parser.AddModelFromFile("../robot_models/Planar-v2.sdf", 'sdf')
parser.AddModelsFromString(full_2r_directives, '.dmd.yaml')

plant.Finalize()

BODY_NAMES = ['l1', 'l2', 'body']
for body_name in BODY_NAMES:  
    AddMultibodyTriad(plant.GetFrameByName(body_name), scene_graph)

    

visualizer = MeshcatVisualizer.AddToBuilder(
    builder, 
    scene_graph.get_query_output_port(),
    meshcat
)

diagram = builder.Build()
diagram_context = diagram.CreateDefaultContext()
    
diagram.ForcedPublish(diagram_context)





In [9]:
# kuka control mode
plant = MultibodyPlant(time_step=1e-4)
parser = Parser(plant)
parser.AddModelFromFile("../../robot_models/Planar-v2.sdf", 'sdf')
plant.Finalize()

context = plant.CreateDefaultContext()
plant.get_actuation_input_port().FixValue(context, np.zeros(2))

simulator = Simulator(plant, context)
simulator.AdvanceTo(0.5)



In [10]:
default_context = plant.CreateDefaultContext()
mutable_context = plant.GetMyMutableContextFromRoot(default_context)
plant.get_actuation_input_port().FixValue(mutable_context, np.zeros(2))

In [11]:
simulator = Simulator(plant, mutable_context)
simulator.AdvanceTo(5.0)

In [12]:
svg = pydot.graph_from_dot_data(diagram.GetGraphvizString())[0].create_svg()
with open('2r_svg_graph.png', 'wb') as f:
    f.write(svg)

In [13]:

simulator = Simulator(diagram, def_context)
simulator.AdvanceTo(5.0)


NameError: name 'def_context' is not defined